# Continued Data Cleaning Walkthrough: NYC Public Schools


In [1]:
# Code from data_cleaning_walkthrough course

import pandas as pd
import numpy as np
import re

data_files = [
    "ap_2010.csv",
    "class_size.csv",
    "demographics.csv",
    "graduation.csv",
    "hs_directory.csv",
    "sat_results.csv"
]

data = {}

for file in data_files:
    read_file = pd.read_csv("{0}".format(file))
    key_name = file[:-4]
    data[key_name] = read_file
    
all_survey = pd.read_csv("survey_all.txt", delimiter='\t', encoding='windows-1252')
d75_survey = pd.read_csv("survey_d75.txt", delimiter='\t', encoding='windows-1252')
survey = pd.concat([all_survey,d75_survey], axis=0)

survey["DBN"] = survey["dbn"]
survey = survey.loc[:,["DBN", "rr_s", "rr_t", "rr_p", "N_s", "N_t", "N_p", "saf_p_11", "com_p_11", "eng_p_11", "aca_p_11", "saf_t_11", "com_t_11", "eng_t_11", "aca_t_11", "saf_s_11", "com_s_11", "eng_s_11", "aca_s_11", "saf_tot_11", "com_tot_11", "eng_tot_11", "aca_tot_11"]]
data["survey"] = survey


def pad(x):
    if len(str(x)) == 1:
        return str(x).zfill(2)
    else:
        return str(x)
    
data['hs_directory']['DBN'] = data['hs_directory']['dbn']
data['class_size']['padded_csd'] = data['class_size']['CSD'].apply(pad)
data['class_size']['DBN'] = data['class_size']['padded_csd'] + data['class_size']['SCHOOL CODE']


categories = ['SAT Math Avg. Score', 'SAT Critical Reading Avg. Score', 'SAT Writing Avg. Score']
for x in categories:
    data['sat_results'][x] = pd.to_numeric(data['sat_results'][x], errors='coerce')
data['sat_results']['sat_score'] = data['sat_results'][categories[0]] + data['sat_results'][categories[1]] + data['sat_results'][categories[2]]


def lat(x):
    x = re.findall("\(.+\)", x)
    for item in x:
        item = item.replace("(", "")
        item = item.replace(")", "")
        item = item.replace(",", "")
        item = item.split()
        return item[0]

data['hs_directory']['lat'] = data['hs_directory']['Location 1'].apply(lat)


def lon(x):
    x = re.findall("\(.+\)", x)
    for item in x:
        item = item.replace("(", "")
        item = item.replace(")", "")
        item = item.replace(",", "")
        item = item.split()
        return item[1]

data['hs_directory']['lon'] = data['hs_directory']['Location 1'].apply(lon)

data['hs_directory']['lat'] = pd.to_numeric(data['hs_directory']['lat'], errors='coerce')
data['hs_directory']['lon'] = pd.to_numeric(data['hs_directory']['lon'], errors='coerce')

class_size = data['class_size']
print (class_size["GRADE "].unique())



['0K' '01' '02' '03' '04' '05' '0K-09' nan '06' '07' '08' 'MS Core' '09-12'
 '09']


In [2]:
# Only using highschool and GEN ED values from 'GRADE ' and 'PROGRAM TYPE' from class size.


class_size = data['class_size']
class_size = class_size[class_size['GRADE '] == '09-12']
class_size = class_size[class_size['PROGRAM TYPE'] == 'GEN ED']

print (class_size.head())

     CSD BOROUGH SCHOOL CODE                                    SCHOOL NAME  \
225    1       M        M292  Henry Street School for International Studies   
226    1       M        M292  Henry Street School for International Studies   
227    1       M        M292  Henry Street School for International Studies   
228    1       M        M292  Henry Street School for International Studies   
229    1       M        M292  Henry Street School for International Studies   

    GRADE  PROGRAM TYPE CORE SUBJECT (MS CORE and 9-12 ONLY)  \
225  09-12       GEN ED                              ENGLISH   
226  09-12       GEN ED                              ENGLISH   
227  09-12       GEN ED                              ENGLISH   
228  09-12       GEN ED                              ENGLISH   
229  09-12       GEN ED                                 MATH   

    CORE COURSE (MS CORE and 9-12 ONLY) SERVICE CATEGORY(K-9* ONLY)  \
225                           English 9                           -  

In [3]:
import numpy as np

class_size = class_size.groupby('DBN')
class_size = class_size.agg(np.mean)
class_size.reset_index(inplace=True)

data['class_size'] = class_size
print(data["class_size"].head())


      DBN  CSD  NUMBER OF STUDENTS / SEATS FILLED  NUMBER OF SECTIONS  \
0  01M292    1                            88.0000            4.000000   
1  01M332    1                            46.0000            2.000000   
2  01M378    1                            33.0000            1.000000   
3  01M448    1                           105.6875            4.750000   
4  01M450    1                            57.6000            2.733333   

   AVERAGE CLASS SIZE  SIZE OF SMALLEST CLASS  SIZE OF LARGEST CLASS  \
0           22.564286                   18.50              26.571429   
1           22.000000                   21.00              23.500000   
2           33.000000                   33.00              33.000000   
3           22.231250                   18.25              27.062500   
4           21.200000                   19.40              22.866667   

   SCHOOLWIDE PUPIL-TEACHER RATIO  
0                             NaN  
1                             NaN  
2                   

In [4]:
# Filtering through demographics to select most recent year only

data['demographics'] = data['demographics'][data['demographics']['schoolyear'] == 20112012]

print (data['demographics'].head())

       DBN                                              Name  schoolyear  \
6   01M015  P.S. 015 ROBERTO CLEMENTE                           20112012   
13  01M019  P.S. 019 ASHER LEVY                                 20112012   
20  01M020  PS 020 ANNA SILVER                                  20112012   
27  01M034  PS 034 FRANKLIN D ROOSEVELT                         20112012   
35  01M063  PS 063 WILLIAM MCKINLEY                             20112012   

   fl_percent  frl_percent  total_enrollment prek    k grade1 grade2  \
6         NaN         89.4               189   13   31     35     28   
13        NaN         61.5               328   32   46     52     54   
20        NaN         92.5               626   52  102    121     87   
27        NaN         99.7               401   14   34     38     36   
35        NaN         78.9               176   18   20     30     21   

      ...     black_num black_per hispanic_num hispanic_per white_num  \
6     ...            63      33.3    

In [5]:
# Filtering through 'graduation' and only selecting rows with 'Cohort' of '2006'
# and 'Demographics' equaling 'Total Cohort'.

data['graduation'] = data['graduation'][data['graduation']['Cohort'] == "2006"]
data['graduation'] = data['graduation'][data['graduation']['Demographic'] == 'Total Cohort']

print (data['graduation'].head())


     Demographic     DBN                            School Name Cohort  \
3   Total Cohort  01M292  HENRY STREET SCHOOL FOR INTERNATIONAL   2006   
10  Total Cohort  01M448    UNIVERSITY NEIGHBORHOOD HIGH SCHOOL   2006   
17  Total Cohort  01M450             EAST SIDE COMMUNITY SCHOOL   2006   
24  Total Cohort  01M509                MARTA VALLE HIGH SCHOOL   2006   
31  Total Cohort  01M515  LOWER EAST SIDE PREPARATORY HIGH SCHO   2006   

    Total Cohort Total Grads - n Total Grads - % of cohort Total Regents - n  \
3             78              43                     55.1%                36   
10           124              53                     42.7%                42   
17            90              70                     77.8%                67   
24            84              47                       56%                40   
31           193             105                     54.4%                91   

   Total Regents - % of cohort Total Regents - % of grads  \
3            

In [8]:
# Converting certain columns in 'ap_2010' to numeric

cols = ['AP Test Takers ', 'Total Exams Taken', 'Number of Exams with scores 3 4 or 5']


for item in cols:
    data['ap_2010'][item] = pd.to_numeric(data['ap_2010'][item], errors="coerce")

In [10]:
# Using left combine to combine 'sat_results' with 'ap_2010' and 'demographics'

combined = data["sat_results"]
combined = combined.merge(data['ap_2010'], on="DBN", how="left")
combined = combined.merge(data['graduation'], on="DBN", how="left")

print (combined.head())
print (combined.shape)

      DBN                                    SCHOOL NAME  \
0  01M292  HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES   
1  01M448            UNIVERSITY NEIGHBORHOOD HIGH SCHOOL   
2  01M450                     EAST SIDE COMMUNITY SCHOOL   
3  01M458                      FORSYTH SATELLITE ACADEMY   
4  01M509                        MARTA VALLE HIGH SCHOOL   

  Num of SAT Test Takers  SAT Critical Reading Avg. Score  \
0                     29                            355.0   
1                     91                            383.0   
2                     70                            377.0   
3                      7                            414.0   
4                     44                            390.0   

   SAT Math Avg. Score  SAT Writing Avg. Score  sat_score  \
0                404.0                   363.0     1122.0   
1                423.0                   366.0     1172.0   
2                402.0                   370.0     1149.0   
3                401.0      

In [11]:
# Combining the rest of the data sets using "inner" join type.

combined = combined.merge(data['class_size'], on="DBN", how="inner")
combined = combined.merge(data['demographics'], on="DBN", how="inner")
combined = combined.merge(data['survey'], on="DBN", how="inner")
combined = combined.merge(data['hs_directory'], on="DBN", how="inner")

print (combined.head())
print (combined.shape)

      DBN                                        SCHOOL NAME  \
0  01M292      HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES   
1  01M448                UNIVERSITY NEIGHBORHOOD HIGH SCHOOL   
2  01M450                         EAST SIDE COMMUNITY SCHOOL   
3  01M509                            MARTA VALLE HIGH SCHOOL   
4  01M539  NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...   

  Num of SAT Test Takers  SAT Critical Reading Avg. Score  \
0                     29                            355.0   
1                     91                            383.0   
2                     70                            377.0   
3                     44                            390.0   
4                    159                            522.0   

   SAT Math Avg. Score  SAT Writing Avg. Score  sat_score  \
0                404.0                   363.0     1122.0   
1                423.0                   366.0     1172.0   
2                402.0                   370.0     1149.0   
3   

In [12]:
# Filling all NaN and null values with the average of that column.
# If there is no average, fill the space with 0.

means = combined.mean()
combined = combined.fillna(means)
combined = combined.fillna(0)

print (combined.head())

      DBN                                        SCHOOL NAME  \
0  01M292      HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES   
1  01M448                UNIVERSITY NEIGHBORHOOD HIGH SCHOOL   
2  01M450                         EAST SIDE COMMUNITY SCHOOL   
3  01M509                            MARTA VALLE HIGH SCHOOL   
4  01M539  NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...   

  Num of SAT Test Takers  SAT Critical Reading Avg. Score  \
0                     29                            355.0   
1                     91                            383.0   
2                     70                            377.0   
3                     44                            390.0   
4                    159                            522.0   

   SAT Math Avg. Score  SAT Writing Avg. Score  sat_score  \
0                404.0                   363.0     1122.0   
1                423.0                   366.0     1172.0   
2                402.0                   370.0     1149.0   
3   

In [13]:
# Creating a new column 'school_dist' with the first two digist of 'DBN'

def district(x):
    return x[0:2]

combined['school_dist'] = combined['DBN'].apply(district)

print (combined['school_dist'].head())

0    01
1    01
2    01
3    01
4    01
Name: school_dist, dtype: object
